In [132]:
import numpy as np
import cv2

In [133]:
# parser = argparse.ArgumentParser()
# parser.add_argument('-i', '--input', help='path to the input video',
#                     required=True)
# parser.add_argument('-c', '--consecutive-frames', default=4, type=int,
#                     dest='consecutive_frames', help='path to the input video')
# args = vars(parser.parse_args())

In [134]:
def get_background(file_path):
    cap = cv2.VideoCapture(file_path)
    # we will randomly select 50 frames for the calculating the median
    frame_indices = cap.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=50)
    frame_rate=cap.get(cv2.CAP_PROP_FPS)
    # we will store the frames in array
    frames = []
    for idx in frame_indices:
        # set the frame id to read that particular frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        frames.append(frame)
    # calculate the median
    median_frame = np.median(frames, axis=0).astype(np.uint8)
    return median_frame,frame_rate

In [135]:
arg_input='input/video_1.mp4'
consecutive_frame=8

In [136]:
# get the background model
background,frame_rate = get_background(arg_input)
# convert the background model to grayscale format
background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
frame_count=0


In [137]:
cap = cv2.VideoCapture(arg_input)
# get the video frame height and width
frame_width = int(cap.get(3)) # width  = vcap.get(cv2.cv.CV_CAP_PROP_FRAME_WIDTH)   # float `width`
frame_height = int(cap.get(4)) # height = vcap.get(cv2.cv.CV_CAP_PROP_FRAME_HEIGHT)  # float `height`
save_name = f"outputs/{arg_input.split('/')[-1]}"
# define codec and create VideoWriter object

codec='mp4v'
fps= frame_rate
out = cv2.VideoWriter(
    save_name,
    cv2.VideoWriter_fourcc(*codec),fps, 
    (frame_width, frame_height)
)

In [138]:

while (cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        frame_count += 1
        orig_frame = frame.copy()
        # IMPORTANT STEP: convert the frame to grayscale first
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if frame_count % consecutive_frame == 0 or frame_count == 1:
            frame_diff_list = []
        # find the difference between current frame and base frame
        frame_diff = cv2.absdiff(gray, background)
        # thresholding to convert the frame to binary
        ret, thres = cv2.threshold(frame_diff, 50, 255, cv2.THRESH_BINARY)
        # dilate the frame a bit to get some more white area...
        # ... makes the detection of contours a bit easier
        dilate_frame = cv2.dilate(thres, None, iterations=2)
        # append the final result into the `frame_diff_list`
        frame_diff_list.append(dilate_frame)
        # if we have reached `consecutive_frame` number of frames
        
        
        
        if len(frame_diff_list) == consecutive_frame:
        
            # add all the frames in the `frame_diff_list`
            sum_frames = sum(frame_diff_list)
            # find the contours around the white segmented areas
            contours, hierarchy = cv2.findContours(sum_frames, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            # draw the contours, not strictly necessary
            #for i, cnt in enumerate(contours):
                #cv2.drawContours(frame, contours, i, (0, 0, 255), 3)
            #cv2.imshow('Detected Objects', frame)
            #print(frame_count)
            #if frame_count+1==2*consecutive_frame:
                #print(frame_count)
                #contours_prev=contours
                #continue
            #print(len(contours_prev))
            #print(len(contours))
            for i,contour in enumerate(contours):
    
                # continue through the loop if contour area is less than 500...
                # ... helps in removing noise detection
                if cv2.contourArea(contour) < 500:
                    continue
                # get the xmin, ymin, width, and height coordinates from the contours
                (x_prev, y_prev, w_prev, h_prev) = cv2.boundingRect(contours_prev[i])
                (x, y, w, h) = cv2.boundingRect(contour)
                
                cv2.putText(orig_frame, 'Moving', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
                
                # draw the bounding boxes
                cv2.rectangle(orig_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                #if x==x_prev:
                    #cv2.putText(orig_frame, 'Static', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
                #elif x>x_prev:
                    #cv2.putText(orig_frame, 'Closer', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
                #else:
                    #cv2.putText(orig_frame, 'Away', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
                
            
            contours_prev=contours
            cv2.imshow('Detected Objects', orig_frame)
            #out.write(orig_frame)
            if cv2.waitKey(100) & 0xFF == ord('q'):
                break
    else:
        break
cap.release()
cv2.destroyAllWindows()

IndexError: tuple index out of range